## private score : 110

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)

# Warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
age_gender_info = pd.read_csv('data/age_gender_info.csv')

## 연령비율 변수 생성

`0~19세_비율` `20~39세_비율` `40~69세_비율` `70세이상_비율`

In [3]:
age_col = [str(x) + '대' for x in range(10,110,10)]
age_col = ['10대미만'] + age_col

In [4]:
for col in age_col:
    W = '(여자)' ;M = '(남자)'
    age_gender_info[col] = age_gender_info[col+W] + age_gender_info[col+M]
    age_gender_info.drop([col+W,col+M],axis=1,inplace=True)

In [5]:
age_gender_info['0~19세_비율'] = age_gender_info.iloc[:,1:3].sum(axis=1)
age_gender_info['20~39세_비율'] = age_gender_info.iloc[:,3:5].sum(axis=1)
age_gender_info['40~69세_비율'] = age_gender_info.iloc[:,5:8].sum(axis=1)
age_gender_info['70세이상_비율'] = age_gender_info.iloc[:,8:].sum(axis=1)
age_gender_info = age_gender_info.loc[:,['지역','0~19세_비율', '20~39세_비율','40~69세_비율', '70세이상_비율']]

In [6]:
train = pd.merge(train,age_gender_info)
test = pd.merge(test,age_gender_info)

*상가 데이터는 제외*

In [7]:
train = train[train['임대건물구분']=='아파트']
test = test[test['임대건물구분']=='아파트']

### NA 처리

`임대료` `임대보증금` : 0 으로 채우기

In [8]:
train.loc[train['임대보증금']=='-','임대보증금'] = 0
train.loc[train['임대료']=='-','임대료'] = 0

In [9]:
test.loc[test['임대보증금']=='-','임대보증금'] = 0
test.loc[test['임대료']=='-','임대료'] = 0

In [10]:
train['임대보증금'] = train['임대보증금'].fillna(0)
test['임대보증금'] = test['임대보증금'].fillna(0)

In [11]:
train['임대료'] = train['임대료'].fillna(0)
test['임대료'] = test['임대료'].fillna(0)

In [12]:
train[['임대료','임대보증금']] = train[['임대료','임대보증금']].astype('int')
test[['임대료','임대보증금']] = test[['임대료','임대보증금']].astype('int')

### 전용면적 단위 통일
ex. `전용면적` : 20-30인 경우 20으로 통일

In [13]:
train['전용면적'] = train['전용면적']//10 * 10
train['전용면적'] = train['전용면적'].astype('int').astype('str')

In [14]:
test['전용면적'] = test['전용면적']//10 * 10
test['전용면적'] = test['전용면적'].astype('int').astype('str')

In [15]:
not_unique = ['자격유형','임대보증금','임대료','전용면적별세대수','전용면적','공급유형']

### 데이터 재구성 및 파생변수 생성

`단지코드`, `전용면적`, `공급유형` 기준으로 데이터 일차원으로 병합

*면적 별로 (20,30) 임대료, 임대보증금의 차이가 커서 단순 평균내는 것은 정보손실 우려가 있음*


`임대료` `임대보증금` 

- NA를 제외하고 평균값 구하기


- `단지코드` `전용면적` `공급유형`를 기준으로 NA 발생시, 지역 별 `전용면적` `공급유형` 기준 `임대료` `임대보증금` 값으로 채움


In [16]:
tmp = train.groupby(['단지코드','전용면적','공급유형'],as_index=False).agg({'전용면적별세대수':'sum'})
tmp = pd.merge(tmp,train[train['임대료']!=0].groupby(['단지코드','전용면적','공급유형'],as_index=False).agg({'임대료':'mean','임대보증금':'mean'}),how='left')
tmp['임대료'].fillna(0,inplace=True)
tmp['임대보증금'].fillna(0,inplace=True)
train = pd.merge(train.drop(not_unique,axis=1).drop_duplicates(),tmp)

tmp = test.groupby(['단지코드','전용면적','공급유형'],as_index=False).agg({'전용면적별세대수':'sum'})
tmp = pd.merge(tmp,test[test['임대료']!=0].groupby(['단지코드','전용면적','공급유형'],as_index=False).agg({'임대료':'mean','임대보증금':'mean'}),how='left')
tmp['임대료'].fillna(0,inplace=True)
tmp['임대보증금'].fillna(0,inplace=True)
test = pd.merge(test.drop(not_unique,axis=1).drop_duplicates(),tmp)

In [17]:
# na처리
for col in ['임대료','임대보증금']:
    tmp = train.groupby(['지역','전용면적','공급유형'],as_index=False)[col].mean().rename(columns={col:'지역별_'+col})
    train = pd.merge(train,tmp)
    test = pd.merge(test,tmp)
    train[col] = train[col].fillna(train['지역별_'+col])
    test[col] = test[col].fillna(train['지역별_'+col])

- `전용면적별세대수_비율` : 아파트 총세대수와 해당 면적에 속한 세대수 비율


- `단지내주차면수_비율` , `등록차량수_비율` : `전용면적별세대수_비율`을 곱해서 새롭게 변수 생성 


    - 한 아파트 안에서 `전용면적` `공급유형`이 다양하게 존재하고, 이에 따라 `임대료` `임대보증금`이 다르기 때문에 단지코드 별 여러개의 row 존재

    - 등록차량수는 아파트 기준이기 때문에 모두 동일한 값으로 나타나져 있음 $$\rightarrow$$ 학습에 방해가 된다고 판단
    
    - 비율을 곱해주어 데이터가 가지고 있는 구조적인 문제를 해결하고자 함

In [18]:
train['전용면적별세대수_비율'] = train['전용면적별세대수']/train['총세대수'] 
train['면적별_등록차량수'] = train['등록차량수'] * train['전용면적별세대수_비율']
train['면적별_단지내주차면수'] = train['단지내주차면수'] * train['전용면적별세대수_비율']

In [19]:
test['전용면적별세대수_비율'] = test['전용면적별세대수']/test['총세대수'] 
test['면적별_단지내주차면수'] = test['단지내주차면수'] * test['전용면적별세대수_비율']

- `인구` : `전용면적별세대수_비율`을 곱해서 해당 아파트의 전용면적 별 인구분포를 살펴보고자 함

In [20]:
age_ratio = ['0~19세_비율', '20~39세_비율','40~69세_비율', '70세이상_비율']
age_col = []
for col in age_ratio:
    col_ = col.split('_')[0] + '_인구수'
    age_col.append(col_)
    train[col_] = train[col] * train['전용면적별세대수']
    test[col_] = test[col] * test['전용면적별세대수']

In [21]:
train.drop(age_ratio,axis=1,inplace=True)
test.drop(age_ratio,axis=1,inplace=True)

- `공가비율` 

In [22]:
train['공가비율'] = train['공가수'] / train['총세대수']
test['공가비율'] = test['공가수'] / test['총세대수']

### 이상치제거

In [23]:
train = train[train['단지코드']!='C1722']
train = train.reset_index(drop=True)

### 카테고리변수 처리

`공급유형` `전용면적` : dummy 변수 생성

In [24]:
train = pd.concat([train,pd.get_dummies(train['공급유형'])],axis=1)
train = pd.concat([train,pd.get_dummies(train['전용면적'],prefix='면적')],axis=1)

In [25]:
test = pd.concat([test,pd.get_dummies(test['공급유형'])],axis=1)
test = pd.concat([test,pd.get_dummies(test['전용면적'],prefix='면적')],axis=1)

In [26]:
for col in ['공공분양', '공공임대(5년)', '장기전세', '면적_60']:
    test[col] = 0

# 모델링

In [27]:
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor

In [28]:
col = ['면적별_단지내주차면수','공가비율','총세대수','임대료','임대보증금']
col = col + age_col

In [29]:
col = col + ['공공분양', '공공임대(10년)',
       '공공임대(50년)', '공공임대(5년)', '공공임대(분납)', '국민임대', '영구임대', '장기전세', '행복주택']
col = col + ['면적_10', '면적_20', '면적_30', '면적_40', '면적_50', '면적_60', '면적_70', '면적_80']

In [30]:
train[col].head(2)

,면적별_단지내주차면수,공가비율,총세대수,임대료,임대보증금,0~19세_인구수,20~39세_인구수,40~69세_인구수,70세이상_인구수,공공분양,공공임대(10년),공공임대(50년),공공임대(5년),공공임대(분납),국민임대,영구임대,장기전세,행복주택,면적_10,면적_20,면적_30,면적_40,면적_50,면적_60,면적_70,면적_80
0,235.916667,0.042222,900,103680.0,15667000.0,26.977756,34.413118,68.858992,149.0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
1,208.303797,0.042194,474,72080.0,9731000.0,47.799514,60.973579,122.005194,264.0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0


In [31]:
lasso = Lasso()
lasso.fit(train[col],train['면적별_등록차량수'])
test['pred_lasso'] = lasso.predict(test[col])

In [32]:
cb = CatBoostRegressor(random_state=2021,verbose=False)
cb.fit(train[col],train['면적별_등록차량수'])
test['pred_cb'] = cb.predict(test[col])

In [33]:
rf = RandomForestRegressor(random_state=2021)
rf.fit(train[col],train['면적별_등록차량수'])
test['pred_rf'] = rf.predict(test[col])

In [34]:
test['num'] = (test['pred_cb'] + test['pred_rf'] + test['pred_lasso'])/3
test.rename(columns={'단지코드':'code'},inplace=True)

In [35]:
test[['pred_rf','pred_cb','pred_lasso','num']].head(2)

,pred_rf,pred_cb,pred_lasso,num
0,106.026758,99.337188,87.613560,97.659169
1,356.566658,357.416605,337.425046,350.469436


### 제출

In [36]:
submit = pd.read_csv('data/sample_submission.csv')
submit.head(2)

,code,num
0,C1072,0
1,C1128,0


In [37]:
submit = pd.merge(submit.drop('num',axis=1),test.groupby(['code'])[['num']].sum().reset_index(),how='left')
submit.loc[submit['code'].isin(['C1327','C2335','C2675']),'num'] = 0

In [38]:
submit.head(2)

,code,num
0,C1072,704.049361
1,C1128,1212.629403
